Risk
====

In [1]:
from collections import namedtuple
import data as datalib
import puma as tw

from tomahawk import risk

In [2]:
# setup the market data
db_credentials = credentials('test', 'localhost', prefix='db_')
csvdf = datalib.CsvDataFeed("c:/rmbaries/git/pifq/src/puma/data/tests/inst/csv_data_feed", source_name='test_csv_data_feed')
csvdh = datalib.HistoricalDataManager(csvdf, **db_credentials)
csvdl = datalib.LiveDataManager(csvdf, **db_credentials)
mdm = datalib.MarketDataManager(csvdh, csvdl)

In [3]:
# setup the Strategy, Portfolio and PositionManager
oms = tw.OrderManager('test_unit', None)
pm = tw.PositionManager('test_unit', oms, None)
port = tw.Portfolio('port_test', oms, pm)
obj_bridge = namedtuple('OB', 'order_manager, portfolio, market_data_manager')(oms, port, mdm)
strat = tw.strategy.ExampleStrategy('TEST1', obj_bridge)
port.add_strategy(strat)

In [4]:
# initialize Risk

rk = risk.Risk(oms)

In [5]:
# insert symbols into strategy
strat.add_symbols([('stock', 'test.sym.1', '1min'), ('stock', 'test.sym.2', '1min')])

In [6]:
# create some orders and have Portfolio process them

oms.market_state('stock', True)
strat.order('stock', 'test.sym.1', 'B', 1000, 'LIMIT', 100.5)
strat.order('stock', 'test.sym.2', 'S', 55, 'LIMIT', 5.5)
port.process_orders()

# see the orders in OrderManager
oms.open_orders_df().show()

  index  Order__uuid                           state    broker_order_id    buy_sell    event_type    CREATED                           originator_id    fill_price    create_timestamp                  symbol      fill_quantity    type    strategy_uuid                         closed    details           booked      quantity  STAGED                            portfolio_id    originator_uuid                       strategy_id    commission    exchange_order_id    product_type    portfolio_uuid
-------  ------------------------------------  -------  -----------------  ----------  ------------  --------------------------------  ---------------  ------------  --------------------------------  ----------  ---------------  ------  ------------------------------------  --------  ----------------  --------  ----------  --------------------------------  --------------  ------------------------------------  -------------  ------------  -------------------  --------------  ---------------------------

In [7]:
# process the orders in the staged order queue of the Portfolio

rk.process_portfolio_orders(port)

In [8]:
# see the state of the orders

oms.open_orders_df().show()

  index  Order__uuid                           state          broker_order_id    buy_sell    event_type    CREATED                           originator_id    fill_price    create_timestamp                  symbol      fill_quantity    type    strategy_uuid                         closed    RISK_ACCEPTED                     details         booked      quantity  STAGED                            portfolio_id    originator_uuid                       strategy_id    commission    exchange_order_id    product_type    portfolio_uuid
-------  ------------------------------------  -------------  -----------------  ----------  ------------  --------------------------------  ---------------  ------------  --------------------------------  ----------  ---------------  ------  ------------------------------------  --------  --------------------------------  --------------  --------  ----------  --------------------------------  --------------  ------------------------------------  -------------  --

In [9]:
# and closed orders

oms.closed_orders_df().show()

  index  Order__uuid                           state          broker_order_id    buy_sell    event_type    CREATED                           originator_id    fill_price    create_timestamp                  symbol      fill_quantity    type    strategy_uuid                         closed    details           booked    RISK_REJECTED                       quantity  STAGED                            portfolio_id    originator_uuid                       strategy_id    commission    exchange_order_id    product_type    portfolio_uuid
-------  ------------------------------------  -------------  -----------------  ----------  ------------  --------------------------------  ---------------  ------------  --------------------------------  ----------  ---------------  ------  ------------------------------------  --------  ----------------  --------  --------------------------------  ----------  --------------------------------  --------------  ------------------------------------  -------------

In [11]:
# print the first accepted order
oms.orders_list({'state': 'RISK_ACCEPTED'})[0].print()

uuid         : 54c4cbb3-972e-4549-b5f3-d7cae918d472
state        : RISK_ACCEPTED
originator_id: strategy.TEST1
strategy_id  : TEST1
symbol       : test.sym.2
buy/sell     : sell
quantity     : 55
order_type   : LIMIT
details      : {'price': 5.5}
broker_id    : None
exchange_id  : None
fill_price   : None
fill_qty     : None
commission   : None
booked       : None
closed       : False

state history:
  index  timestamp                         state
-------  --------------------------------  -------------
      0  2017-05-02 12:47:45.062792+00:00  CREATED
      1  2017-05-02 12:47:45.062792+00:00  STAGED
      2  2017-05-02 12:47:45.094004+00:00  RISK_ACCEPTED

replacements :
  index    quantity  details
-------  ----------  --------------
      0          55  {'price': 5.5}

fills        :
index    timestamp    bartime    quantity    price    commission    booked
-------  -----------  ---------  ----------  -------  ------------  --------


In [12]:
# print the first rejected order
oms.orders_list({'state': 'RISK_REJECTED'})[0].print()

uuid         : fe56852c-600d-4dc8-b338-abcc2ba7fa46
state        : RISK_REJECTED
originator_id: strategy.TEST1
strategy_id  : TEST1
symbol       : test.sym.1
buy/sell     : buy
quantity     : 1000
order_type   : LIMIT
details      : {'price': 100.5}
broker_id    : None
exchange_id  : None
fill_price   : None
fill_qty     : None
commission   : None
booked       : None
closed       : True

state history:
  index  timestamp                         state
-------  --------------------------------  -------------
      0  2017-05-02 12:47:45.062792+00:00  CREATED
      1  2017-05-02 12:47:45.062792+00:00  STAGED
      2  2017-05-02 12:47:45.094004+00:00  RISK_REJECTED

replacements :
  index    quantity  details
-------  ----------  ----------------
      0        1000  {'price': 100.5}

fills        :
index    timestamp    bartime    quantity    price    commission    booked
-------  -----------  ---------  ----------  -------  ------------  --------
